# PxW

Testing our wrapper for the very first time. Can I implement this fast?

In [9]:
import prefect
import PxW 

from prefect import Task, Flow

In [2]:
dir(PxW)

['Weka',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'core',
 'dist_name']

In [5]:
weka = PxW.Weka()

In [55]:
weka.fit()

In [68]:
weka_cmd = """java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar"""

tree = "weka.classifiers.trees.J48"
tree_cmd = """{} -C 0.25 -M 2""".format(tree)

model_fn = "tree.model"
model_cmd = """-d {}""".format(model_fn)

data_fn = "../data/raw/iris.arff"
data_cmd = "-t {}".format(data_fn)

cmd = """{} {} {} -no-cv {}""".format(weka_cmd, tree_cmd, data_cmd, model_cmd)

shell = ShellTask()

with Flow("fit") as f:
    fit = shell(command=cmd)

status = f.run()

[2019-08-22 15:30:41,456] INFO - prefect.FlowRunner | Beginning Flow run for 'fit'
[2019-08-22 15:30:41,458] INFO - prefect.FlowRunner | Starting flow run.
[2019-08-22 15:30:41,464] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-08-22 15:30:41,466] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-08-22 15:30:41,467] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-08-22 15:30:41,802] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-08-22 15:30:41,803] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


In [86]:
print(status.result[fit].result.decode("utf-8"))


Options: -C 0.25 -M 2 

=== Classifier model (full training set) ===

J48 pruned tree
------------------

petal width (cm) <= 0.6: 0.0 (50.0)
petal width (cm) > 0.6
|   petal width (cm) <= 1.7
|   |   petal length (cm) <= 4.9: 1.0 (48.0/1.0)
|   |   petal length (cm) > 4.9
|   |   |   petal width (cm) <= 1.5: 2.0 (3.0)
|   |   |   petal width (cm) > 1.5: 1.0 (3.0/1.0)
|   petal width (cm) > 1.7: 2.0 (46.0/1.0)

Number of Leaves  : 	5

Size of the tree : 	9


Time taken to build model: 0.15 seconds

Time taken to test model on training data: 0 seconds

=== Error on training data ===

Correctly Classified Instances         147               98      %
Incorrectly Classified Instances         3                2      %
Kappa statistic                          0.97  
Mean absolute error                      0.0233
Root mean squared error                  0.108 
Relative absolute error                  5.2482 %
Root relative squared error             22.9089 %
Total Number of Instances      

In [53]:
from prefect import Flow
from prefect.tasks.shell import ShellTask

task = ShellTask()

with Flow("testrun") as f:
    weka_cmd = ""
    tree_cmd = "weka.classifiers.trees.J48 -C 0.25 -M 2"
    data_fnm = "../data/raw/iris.arff"
    
    output = """
    -classifications "weka.classifiers.evaluation.output.prediction.CSV -p first"
    """ '
    
    
    cmd = "{} {} -t {}".format(weka_cmd, tree_cmd, data_fnm)
    pwd = task(command=cmd)

state = f.run()

SyntaxError: EOL while scanning string literal (<ipython-input-53-208a854ed353>, line 13)

In [54]:
cmd

'java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar weka.classifiers.trees.J48 -C 0.25 -M 2 -t ../data/raw/iris.arff'

In [52]:
state.result[pwd].result

b'\nOptions: -C 0.25 -M 2 \n\n=== Classifier model (full training set) ===\n\nJ48 pruned tree\n------------------\n\npetal width (cm) <= 0.6: 0.0 (50.0)\npetal width (cm) > 0.6\n|   petal width (cm) <= 1.7\n|   |   petal length (cm) <= 4.9: 1.0 (48.0/1.0)\n|   |   petal length (cm) > 4.9\n|   |   |   petal width (cm) <= 1.5: 2.0 (3.0)\n|   |   |   petal width (cm) > 1.5: 1.0 (3.0/1.0)\n|   petal width (cm) > 1.7: 2.0 (46.0/1.0)\n\nNumber of Leaves  : \t5\n\nSize of the tree : \t9\n\n\nTime taken to build model: 0.15 seconds\n\nTime taken to test model on training data: 0 seconds\n\n=== Error on training data ===\n\nCorrectly Classified Instances         147               98      %\nIncorrectly Classified Instances         3                2      %\nKappa statistic                          0.97  \nMean absolute error                      0.0233\nRoot mean squared error                  0.108 \nRelative absolute error                  5.2482 %\nRoot relative squared error             22.

In [30]:
from prefect import Flow
from prefect.tasks.shell import ShellTask
from prefect.utilities.debug import raise_on_exception

task = ShellTask(helper_script="java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar")

command = "weka.classifiers.trees.J48 -C 0.25 -M 2 -t iris.arff"

with Flow("My Flow") as f:
    # both tasks will be executed in home directory
    train_tree = task(command=command)
    
with raise_on_exception():
    out = f.run()

[2019-08-22 14:20:56,639] INFO - prefect.FlowRunner | Beginning Flow run for 'My Flow'
[2019-08-22 14:20:56,641] INFO - prefect.FlowRunner | Starting flow run.
[2019-08-22 14:20:56,644] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-08-22 14:20:56,645] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-08-22 14:20:56,645] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-08-22 14:20:56,719] ERROR - prefect.FlowRunner | Unexpected error while running flow: FAIL('Command failed with exit code 127: b\'Usage: java [-options] class [args...]\\n           (to execute a class)\\n   or  java [-options] -jar jarfile [args...]\\n           (to execute a jar file)\\nwhere options include:\\n    -d32\\t  use a 32-bit data model if available\\n    -d64\\t  use a 64-bit data model if available\\n    -server\\t  to select the "server" VM\\n    -zero\\t  to select the "zero" VM\\n    -dcevm

FAIL: Command failed with exit code 127: b'Usage: java [-options] class [args...]\n           (to execute a class)\n   or  java [-options] -jar jarfile [args...]\n           (to execute a jar file)\nwhere options include:\n    -d32\t  use a 32-bit data model if available\n    -d64\t  use a 64-bit data model if available\n    -server\t  to select the "server" VM\n    -zero\t  to select the "zero" VM\n    -dcevm\t  to select the "dcevm" VM\n                  The default VM is server,\n                  because you are running on a server-class machine.\n\n\n    -cp <class search path of directories and zip/jar files>\n    -classpath <class search path of directories and zip/jar files>\n                  A : separated list of directories, JAR archives,\n                  and ZIP archives to search for class files.\n    -D<name>=<value>\n                  set a system property\n    -verbose:[class|gc|jni]\n                  enable verbose output\n    -version      print product version and

In [19]:
out.message

'Some reference tasks failed.'

In [28]:
out.is_failed()

True